In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Introduction


For the past few months, I have been completing the Google Data Analytics Professional Certificate offered on Coursera. I have been learning about the six stages of the data analysis process (ask, prepare, process, analyze, share, & act), and how to use technical tools (SQL, Tableau, R) in each stage to ultimately gather insights from large amounts of data. To finish the course, Google recommends that each student creates their own Capstone Project to highlight the technical skills they learned during the course and to showcase their understanding of each step of the data analysis process. As such, I have completed my Capstone Project on the Cyclistic Case Study.


## About the Company

Urška Sršen and Sando Mur founded Bellabeat, a high-tech company that manufactures health-focused smart products.
Sršen used her background as an artist to develop beautifully designed technology that informs and inspires women around
the world. Collecting data on activity, sleep, stress, and reproductive health has allowed Bellabeat to empower women with
knowledge about their own health and habits. Since it was founded in 2013, Bellabeat has grown rapidly and quickly
positioned itself as a tech-driven wellness company for women

## Problem

Sršen knows that an analysis of Bellabeat’s available consumer data would reveal more opportunities for growth. She has
asked the marketing analytics team to focus on a Bellabeat product and analyze smart device usage data in order to gain
insight into how people are already using their smart devices. Then, using this information, she would like high-level
recommendations for how these trends can inform Bellabeat marketing strategy

## Ask

1. What are some trends in smart device usage?
2. How could these trends apply to Bellabeat customers?
3. How could these trends help influence Bellabeat marketing strategy?

## Prepare

To answer this question I will be analysing historical Cyclistic bike trip data for 12 months from September 2021 to August 2022. The data is located Google Analytics Capstone case study and also available here on Kaggle https://www.kaggle.com/datasets/salamibrahim/cyclistic The data is organised in csv format. The data belongs to Cylistic and has been stored properly on its database. It's free of bias. It is credible. It is as well Reliable, Original, Comprehensive, Current and Cited. The license is being shown on the Google Analytics Case study. The data of the users is surely kept private and secure and as well made accessible to the company's team involved. All data is consistent. The data provides a good insight about the riders and members. We can derive useful insights out of it. There are some missing data which would be taken care of during the process stage.



Importing data sets

In total there are 18 csv files. From that this study will be using few datasets that will help with the analysis. Those datasets are following:

    daily_Activity
    daily_calories
    daily_intensities
    daily_steps
    heartrate_seconds
    sleep_day
    weight_log_info



## Process



I will be using R. I'm choosing this because it makes me import the huge CSV files easily, which was also time efficient

In [ ]:
#load libraries
library("tidyverse")
library(lubridate)
library(hms)
library(data.table)
library(scales)
library(ggplot2)



Loading all required data. There are about 18 datasets in this case study, but I'll be using some for my analysis


Loading the datasets

In [ ]:
daily_activity <- read_csv("C:/Users/CHRYS/Downloads/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv")
hourly_calories <- read_csv("C:/Users/CHRYS/Downloads/Fitabase Data 4.12.16-5.12.16/hourlyCalories_merged.csv")
hourly_intensities <- read_csv("C:/Users/CHRYS/Downloads/Fitabase Data 4.12.16-5.12.16/hourlyIntensities_merged.csv")
daily_steps <- read_csv("C:/Users/CHRYS/Downloads/Fitabase Data 4.12.16-5.12.16/dailySteps_merged.csv")
hourly_steps <- read_csv("C:/Users/CHRYS/Downloads/Fitabase Data 4.12.16-5.12.16/hourlySteps_merged.csv")
sleep_day <- read_csv("C:/Users/CHRYS/Downloads/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv")
weight <- read_csv("C:/Users/CHRYS/Downloads/Fitabase Data 4.12.16-5.12.16/weightLogInfo_merged.csv")

#bella <- rbind(daily_activity, calories, intensities, daily_steps, hourly_steps, sleep_day, weight)
#head(bella)

In [ ]:
daily_activity <- daily_activity %>% 
  rename(date = activity_date) %>% 
  mutate(date = as_date(date, format = "%m/%d/%Y"))

hourly_calories <- hourly_calories %>% 
  mutate(activity_hour = as.POSIXct(activity_hour, format = "%m/%d/%Y %H:%M" , tz=Sys.timezone())) %>%
  separate(activity_hour, into = c('date', 'time'), sep = ' ', remove = TRUE) 

hourly_intensities <- hourly_intensities %>%
  mutate(activity_hour = as.POSIXct(activity_hour, format = "%m/%d/%Y %H:%M" , tz=Sys.timezone())) %>%
  separate(activity_hour, into = c('date', 'time'), sep = ' ', remove = TRUE)

hourly_steps <- hourly_steps %>%
  mutate(activity_hour = as.POSIXct(activity_hour, format = "%m/%d/%Y %H:%M" , tz=Sys.timezone())) %>%
  separate(activity_hour, into = c('date', 'time'), sep = ' ', remove = TRUE)

sleep_day <- sleep_day %>%
  rename(date = sleep_day) %>%
  mutate(date = as_date(date, format = "%m/%d/%Y"))

weight_log <- weight_log %>%
  mutate(date = as.POSIXct(date, format = "%m/%d/%Y %H:%M" , tz=Sys.timezone())) %>%
  separate(date, into = c('date', 'time'), sep = ' ', remove = TRUE)

Merging daily_activity and daily_sleep into a new data set called daily_log using full_join() function with ‘id’ and ‘date’ as the common column. Also, daily_activity has 530 more rows than daily_sleep so zeros will need to fill in the blanks instead of NA so that our daily_activity data doesn’t get cut off where daily_sleep ends (at row 410).

In [ ]:
daily_log <- full_join(daily_activity, daily_sleep, by = c('id', 'date'))
daily_log[is.na(daily_log)] <- 0
str(daily_log)

Merging hourly_calories, hourly_intensities, and hourly_steps into a new data set called hourly_log.hourly_log <- print(list(hourly_steps, hourly_calories, hourly_intensities) %>%
        reduce(full_join, by = c('id', 'date', 'time')))

In [ ]:
hourly_log <- print(list(hourly_steps, hourly_calories, hourly_intensities) %>%
        reduce(full_join, by = c('id', 'date', 'time')))

Adding weekdays to all data sets. This will help delegate individual days of the week to all categories of information.

In [ ]:
daily_log$weekday <- weekdays(as.Date(daily_log$date))
hourly_log$weekday <- weekdays(as.Date(hourly_log$date))
weight_log$weekday <- weekdays(as.Date(weight_log$date))
str(daily_log)

Creating a column called ‘healthy_activity_minutes’ in the data set daily_log by adding ‘very_active_minutes’ and ‘fairly_active_minutes’ together. This will show how many participants exert themselves more and fall into a healthy activity level category.

In [ ]:
daily_log$healthy_activity_minutes <- c(daily_log$very_active_minutes + daily_log$fairly_active_minutes)

head(daily_log)

Creating a data set for totals by individual Id number. This will show some statistics for each individual participant for the total duration that data was submitted (31 days).

In [ ]:
totals_per_id <- daily_log %>%
  group_by(id) %>%
  summarise(total_steps_per_id = sum(total_steps), total_healthy_activity_minutes = sum(very_active_minutes + fairly_active_minutes), total_days_tracked = sum(total_steps > 0), total_nights_tracked = sum(total_minutes_asleep > 0))
str(totals_per_id)

Creating a data set called weekday_log to see average daily steps based on the individual day of the week. This will show us the participants as a group for the average steps taken for each day of the week.

In [ ]:
weekday_log <- daily_log %>%
  mutate(weekday = weekdays(date))

weekday_log$weekday <-ordered(weekday_log$weekday, levels=c("Monday", "Tuesday", "Wednesday", "Thursday",
"Friday", "Saturday", "Sunday"))

 weekday_log <-weekday_log %>%
  group_by(weekday) %>%
  summarize (average_daily_steps = mean(total_steps))

head(weekday_log)

After creating new data sets some trends I noticed were: some days of the week were more popular than others for participants tracking steps, half of the participants engaged in healthy activity minutes, some participants hardly tracked anything, and very few participants tracked their weight.

My goal was to find trends in which categories participants tracked the most to see where their interests are when it comes to having a fitness tracker. Needing this information is imperative to being able to provide recommendations for Bellabeat.

The visualizations I created include information about what participants tracked, what was most important to them. To give recommendations to Bellabeat I would first want to know why participants would purchase a fitness tracker in the first place. What features in the fitness tracker are the most popular?




In [ ]:
totals_per_id$id <-as.character(totals_per_id$id)
ggplot(data = totals_per_id) +
  geom_col(mapping = aes(id, total_days_tracked, fill =total_days_tracked))+
  theme(axis.text.x = element_text(angle = 90))+
  scale_fill_gradient(low = "plum1", high = "royalblue3")+
  labs(title = "The number of days participants tracked",
       subtitle="Days Tracked by Individual ID", x="ID", y="Number of days tracked")

In [ ]:
#Nights Tracked

totals_per_id$id <-as.character(totals_per_id$id)
ggplot(data = totals_per_id) +
  geom_col(mapping = aes(id, total_nights_tracked, fill =total_nights_tracked))+
  theme(axis.text.x = element_text(angle = 90))+
  scale_fill_gradient(low = "dodgerblue3", high = "mediumaquamarine")+
  labs(title = "The number of nights participants tracked",
       subtitle="Nights Tracked by Individual ID", x="ID", y="Number of nights tracked")

In [ ]:
#Weight Tracked

weight_tracked %>% 
ggplot(aes(x="",y=percent, fill=participants)) +
  geom_bar(stat = "identity", width = 1)+
  coord_polar("y", start=0)+
  theme_minimal()+
  theme(axis.title.x= element_blank(),
        axis.title.y = element_blank(),
        panel.border = element_blank(), 
        panel.grid = element_blank(), 
        axis.ticks = element_blank(),
        axis.text.x = element_blank(),
        plot.title = element_text(hjust = 0.5, size=14, face = "bold")) +
  scale_fill_manual(values = c("royalblue3", "plum")) +
  geom_text(aes(label = percent),
            position = position_stack(vjust = 0.5))+
  labs(title="How many participants tracked weight?")

In [ ]:
#Weekday Average Steps

ggplot(data=weekday_log, aes(x=weekday, y=average_daily_steps))+ 
  geom_bar(stat="identity", fill="plum4")+
  labs(title = "Days of the week participants get the most steps", 
       subtitle = "Average Total Steps Per Day", x="Weekday", y="Average Daily Steps")

This visual shows the average steps per day of the week for the participants as a whole. What’s interesting is that we can see that there are more steps taken on Tuesday and Saturday, then Monday and Wednesday following closely behind. With Sunday having the lowest average steps I would assume that participants use this day to rest and get ready for the work week.

In [ ]:
#Hourly Total Steps

hourly_log %>%
  group_by(time) %>%
  summarize(average_steps = mean(step_total)) %>%
  ggplot() +
  geom_col(mapping = aes(x=time, y = average_steps, fill = average_steps)) + 
  labs(title = "Most active time of day for participants", 
       subtitle = "Average Total Steps Per Hour", x="Hour", y="Steps") + 
  scale_fill_gradient(low = "blue", high = "pink")+
  theme(axis.text.x = element_text(angle = 90))

In [ ]:
#Steps & Intensity

ggplot(data = hourly_log)+
  geom_point(mapping = aes(x = step_total, y = total_intensity, color = id),)+
  geom_smooth(mapping = aes(x = step_total, y = total_intensity))+
  labs(title="Correlation between steps and intensity levels", 
       subtitle="Total steps & total intensity each hour by Id", 
       x="Total Steps", y="Total Intensity")

In [ ]:
#Healthy Activity Minutes

totals_per_id$id <-as.character(totals_per_id$id)
ggplot(data = totals_per_id) +
  geom_col(mapping = aes(id, total_healthy_activity_minutes, fill =total_healthy_activity_minutes))+
  theme(axis.text.x = element_text(angle = 90))+
  scale_fill_gradient(low = "slateblue2", high = "indianred2")+
  labs(title = "Participants total healthy activity minutes for 31 days",
       subtitle="Very active minute + fairly active minutes", x="ID", y="Healthy Activity Minutes")

 Conclusion (Act Phase)

My analysis and visuals shows that participants wore their fitness tracker during the day to track steps. Counting steps is a base feature that leads to other statistics such as calories lost, distance traveled and weight loss.

76% of participants did not track their weight, more than 50% of participants did not track their sleep for more than 10 days, and more than 60% of participants did not engaged in healthy activity minutes exceeding 1,000 minutes a month. Based on these statistics, tracking steps is what’s important to participants.

Bellabeat can use this information and my analysis to promote their products, focus on promoting what features consumers look for in a fitness tracker, and showcase other features that stand out from the rest. Bellabeat can offer more than what your typical fitness tracker can and in a more comfortable way.


7. Recommendations

Bellabeat is a wellness brand focused on women. With that being said, it is hard to analyze data with so many limitations and not knowing the genders of our participants. This knowledge would help us make better recommendations for Bellabeat. I would recommend that Bellabeat conduct it’s own survey over a longer period of time to produce more accurate data results.


By analyzing the smart device data that was suggested, these are my recommendation for Bellabeat to discover growth opportunities to improve the company and their marketing strategy:


    Target Clientele - Knowing that the consumers of Bellabeat products are young adult women, discover advertising techniques targeting that demographic would be benificial.

    Notifications -

        Sending personalized notifications and messages makes a person feel more relevant and is more likely to be opened. This could be great motivation to perform the task that is asked of them.

        Create a notification that alerts the person when they have been in a sedentary state for a long period of time during the day.

        Remind them of their goals and suggest movement activity

        Set up a weekly notification to record weight and reward them for doing so.

        Promote healthy sleep habits with notifications suggesting winding down techniques, meditation, relaxing music or podcasts.

        Notifications for drinking more water, bonuses if they have Bellabeats smart water bottle.

    Social Media - Having a online community of women share their success stories, achievements, workouts, and meal plans brings more people together. Adding the ability to ask questions anonymously and creating challenges would be important too.

    Challenges - Re-engagement after someone is sick or goes on vacation can be hard for that person to get back into routine. Creating different kinds of challenges can keep users interested and engaged. Continuous motivation is key when it comes to consistently tracking your health. Creating games to keep users maintaining healthy habits and reward them for passing challenges can keep people engaged too.


Promote Bellabeat Products through social media reaching out to groups with women, companies that have fitness programs for employees, fitness facilities, and young moms. Bellabeat has many products that are amazing and not only help women track their health but are comfortable to wear.

Having the option to not wear something on your wrist constantly had me immediately interested. From personal experience, I hated wearing my fitness tracker to bed and I forget it sometimes when I put it on the charger. Having a necklace or clipping something to a shirt is a game changer. I’m sure I’m not alone with this feeling and many other women out there could benefit from knowing these items exist.

Bellabeats smart water bottle and products you don’t have to charge constantly like the Leaf or Time should be highly advertised. I look forward to seeing more of these products on the market soon.

In [ ]:
# convert "ActivityDate" to datatime64 dtype and format to yyyy-mm-dd
daily_activity["ActivityDate"] = pd.to_datetime(daily_activity["ActivityDate"], format="%m/%d/%Y")# re-print information to confirm
daily_activity.info()# print the first 5 rows of "ActivityDate" to confirm
daily_activity["ActivityDate"].head()

I removed rows which contains 0 total steps

In [ ]:
daily_activities1 <- daily_activities %>% 
  filter(TotalSteps!=0)

unique

In [ ]:
n_distinct(daily_activity1$Id)
n_distinct(calories$Id)
n_distinct(intensity_df$Id)
n_distinct(daily_steps$Id)
n_distinct(hourly_steps$Id)
n_distinct(sleep_day$Id)
n_distinct(weight$Id)

Checking for duplicates

In [ ]:
sum(duplicated(daily_activity))
sum(duplicated(calories))
sum(duplicated(intensity))
sum(duplicated(daily_steps))
sum(duplicated(hourly_steps))
sum(duplicated(sleep_day))
sum(duplicated(weight))

Since DailySleep has duplcates, I'll be gettig rid of them

In [ ]:
DailySleep <- DailySleep %>%
  distinct() %>%
  drop_na()

sum(duplicated(DailySleep)) ##We verified that the duplicates have been removed



Formating the date type from 12 hr  to 24 hr format by using strptime function


In [ ]:
Heartrate$Time <- strptime(Heartrate$Time, format = "%m/%d/%Y %I:%M:%S %p")
Intensity_df$ActivityHour <- strptime(Intensity_df$ActivityHour, format = "%m/%d/%Y %I:%M:%S %p")
Calories$ActivityHour <- strptime(Calories$ActivityHour, format = "%m/%d/%Y %I:%M:%S %p")



Finding missing values

In [ ]:


sum(is.na(hourly_calories))
sum(is.na(hourly_steps))
sum(is.na(hourly_intensities))
sum(is.na(sleep_day))
sum(is.na(daily_activity))




sum(is.na(Heartrate)) 
sum(is.na(Intensity_df))
sum(is.na(Calories))
sum(is.na(Steps))
sum(is.na(DailyCalories))
sum(is.na(DailyActivity1))
sum(is.na(DailySleep))
sum(is.na(Sleep_min))



In [ ]:
# Cleaning data

user_id <- as.character(daily_activity$Id)

daily_activity_date <- format(as.Date(daily_activity$ActivityDate), "%m/%d/%y")

daily_activity <- daily_activity %>% 
  mutate(formatted_date = daily_activity_date, weekday_date = weekdays(daily_activity_date))

new_sleep_day <- separate(sleep_day, SleepDay, into = c("Sleep_Date","Sleep_Time"), sep = " ") %>%
  mutate(formatted_date = as.Date(Sleep_Date, format = "%m/%d/%y"))

hourly_calories_hour <- mdy_hms(hourly_calories$ActivityHour)
new_hourly_calories <- hourly_calories %>% 
  mutate(formatted_hour = format(as.POSIXct(hourly_calories_hour),"%H:%M:%S"))

hourly_intensities_hour <- mdy_hms(hourly_intensities$ActivityHour)
new_hourly_intensities <- hourly_intensities %>% 
  mutate(formatted_hour = format(as.POSIXct(hourly_intensities_hour),"%H:%M:%S"))

hourly_steps_hour <- mdy_hms(hourly_steps$ActivityHour)
new_hourly_steps <- hourly_steps %>% 
  mutate(formatted_hour = format(as.POSIXct(hourly_steps_hour),"%H:%M:%S"))

In [ ]:
daily_activity$ActivityDate <- format(as.Date(daily_activity$date), "%A")


In [ ]:
daily_activity 
calories
intensities
daily_steps
hourly_steps 
sleep_day
weight

bike_rides$date <- as.Date(bike_rides$started_at)
bike_rides$month <- format(as.Date(bike_rides$date), "%m")
bike_rides$day <- format(as.Date(bike_rides$date), "%d")
bike_rides$year <- format(as.Date(bike_rides$date), "%Y")
bike_rides$day_of_week <- format(as.Date(bike_rides$date), "%A")




## In the Heartrate, Intensity and Calories dataset the Date time stamp is separated to streamline it with all other datasets 
## because we know that while analysing data, visualizing or merging the sheets, this will come in handy
Heartrate_df <- separate(Heartrate,Time, into = c('Date', 'Time'), sep = ' ') 
Intensity_f <- separate(Intensity_df,ActivityHour, into = c('Date', 'Time'), sep = ' ')
Calories_df <- separate(Calories, ActivityHour, into = c('Date', 'Time'), sep = ' ')
glimpse(Heartrate_df)
glimpse(Intensity_f)
glimpse(Calories_df)



In [ ]:

# convert "ActivityDate" to datatime64 dtype and format to yyyy-mm-dd
daily_activity["ActivityDate"] = pd.to_datetime(daily_activity["ActivityDate"], format="%m/%d/%Y")# re-print information to confirm
daily_activity.info()# print the first 5 rows of "ActivityDate" to confirm
daily_activity["ActivityDate"].head()

In [ ]:
# create new column "day_of_the_week" to represent day of the week 
df_activity["DayOfTheWeek"] = df_activity["ActivityDate"].dt.day_name()

# print 1st 5 rows to confirm
df_activity["DayOfTheWeek"].head(5)

In [ ]:
# create new column "total_mins" containing sum of total minutes.
df_activity["total_mins"] = df_activity["very_active_mins"] + df_activity["fairly_active_mins"] + df_activity["lightly_active_mins"] + df_activity["sedentary_mins"]
df_activity["total_mins"].head(5)

In [ ]:
# create new column *total_hours* by converting to hour and round float to two decimal places
df_activity["total_hours"] = round(df_activity["total_mins"] / 60)# print 1st 5 rows to confirm
df_activity["total_hours"].head(5)

## Analyse

In [ ]:
# pull general statistics
df_activity.describe()

## Share

In [ ]:
# import matplotlib package
import matplotlib.pyplot as plt# plotting histogram
plt.style.use("default")
plt.figure(figsize=(6,4)) # specify size of the chart
plt.hist(df_activity.day_of_the_week, bins = 7, 
         width = 0.6, color = "lightskyblue", edgecolor = "black")# adding annotations and visuals
plt.xlabel("Day of the week")
plt.ylabel("Frequency")
plt.title("No. of times users logged in app across the week")
plt.grid(True)
plt.show()

In [ ]:
# import matplotlib package
import matplotlib.pyplot as plt# plotting scatter plot
plt.style.use("default")
plt.figure(figsize=(8,6)) # specify size of the chart
plt.scatter(df_activity.total_steps, df_activity.calories, 
            alpha = 0.8, c = df_activity.calories, 
            cmap = "Spectral")# add annotations and visuals
median_calories = 2303
median_steps = 7637plt.colorbar(orientation = "vertical")
plt.axvline(median_steps, color = "Blue", label = "Median steps")
plt.axhline(median_calories, color = "Red", label = "Median calories burned")
plt.xlabel("Steps taken")
plt.ylabel("Calories burned")
plt.title("Calories burned for every step taken")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:


In order to get the dataset which will provide this study the hourly activity, three datasets need to be merged together. The final hourly_activity dataset will give clear view for better analysis.

hourly_activity1 <- merge(x = hourly_calories, y = hourly_intensities, by = c("Id", "ActivityHour"))
hourly_activity <- merge(x= hourly_activity1, y = hourly_steps, by = c("Id", "ActivityHour"))



In [ ]:


library(lubridate)
daily_activity$ActivityDate <- as.POSIXct(daily_activity$ActivityDate, format = "%m/ %d/ %Y")
daily_calories$ActivityDay <- as.POSIXct(daily_calories$ActivityDay, format = "%m/ %d/ %Y")
daily_intensities$ActivityDay <- as.POSIXct(daily_intensities$ActivityDay, format = "%m/ %d/ %Y")
daily_steps$ActivityDay <- as.POSIXct(daily_steps$ActivityDay, format = "%m/ %d/ %Y")
heartrate_seconds$Time <- strptime(heartrate_seconds$Time, format = "%m/ %d/ %Y %I:%M:%S %p")
hourly_activity$ActivityHour <- strptime(hourly_activity$ActivityHour, format = "%m/ %d/ %Y %I:%M:%S %p")
sleep_day$SleepDay <- as.POSIXct(sleep_day$SleepDay, format = "%m/ %d/ %Y %I:%M:%S %p")
weight_log_info$Date <- strptime(weight_log_info$Date, format = "%m/ %d/ %Y %I:%M:%S %p")

